In [1]:
import os
import json
from moviepy import (
    AudioFileClip,
    CompositeAudioClip,
    CompositeVideoClip,
    ColorClip,
    TextClip,
    VideoFileClip,
    concatenate_videoclips,
    vfx
)
import re

In [2]:
def demoji(text):
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [3]:
INPUT_FILE = r"./export/result.json"
INPUT_DIR = r"./export/"          # папка с исходными видео
OUTPUT_FILE = r"./result/output.mp4"    # итоговый файл
W, H = 1920, 1080
VIDEO_FONT_PATH = "./static/Roboto-Regular.ttf"
FADE_DURATION = 0.5   # плавный переход между видео

TITLE_FONT_PATH = "./static/Roboto-Bold.ttf"
TITLE_TEXT = "АНТИДОБРОКЕК #1"
INTRO_MUSIC = "./static/intro.mp3"         # музыка для вступления
TITLE_DURATION = 19.98  # секунды

In [4]:
export_dict = {}
with open(INPUT_FILE, 'r') as input_file:
    export_dict = json.load(input_file)

In [5]:
videos = []
for message in export_dict["messages"]:
    if message.get("media_type", "unknown")!= "video_file":
        continue

    title = ""
    for text_entity in message["text_entities"]:
        if text_entity["type"] not in ("plain", "code"):
            continue
        title += text_entity["text"].replace("\n\n", " ").strip()+" "
    video = {
        "filepath": os.path.join(INPUT_DIR, message["file"]),
        "title": demoji(title).strip(),
    }

    videos.append(video)


In [6]:
title_bg = ColorClip((W, H), color=(0, 0, 0), duration=TITLE_DURATION)

# Текст
title_text = TextClip(
    font=TITLE_FONT_PATH,
    text=TITLE_TEXT,
    color="white",
    font_size=60,
    size=(W, int(H * 0.8)),
    text_align="center",
).with_duration(TITLE_DURATION).with_position(("center", "center"))

# Собираем титр
title_clip = CompositeVideoClip([title_bg, title_text])

# Музыка на титры
audio_tracks = []
if os.path.exists(INTRO_MUSIC):
    print("Intro music added")
    intro_music = AudioFileClip(INTRO_MUSIC).with_volume_scaled(0.2).subclipped(0, TITLE_DURATION)
    title_clip = title_clip.with_audio(intro_music)
    audio_tracks.append(intro_music)


Intro music added


In [7]:
video_tracks = [title_clip.with_start(0)]
current_time = TITLE_DURATION

In [8]:
clips = [title_clip]

In [9]:
for video in videos[-3: ]:
    clip = VideoFileClip(video["filepath"])

    # Масштабирование
    clip_resized = clip.resized(height=H)
    if clip_resized.w > W:
        clip_resized = clip_resized.resized(width=W)

    # Подложка
    bg = ColorClip((W, H), color=(0, 0, 0), duration=clip.duration)

    # Подпись к видео — в углу справа
    caption_w = (W-clip_resized.w)//2
    caption = TextClip(
        font=VIDEO_FONT_PATH,
        size=(caption_w, H),
        text=video["title"],
        color="white",
        font_size=20,
    ).with_duration(clip.duration).with_position(("right", "top"))

    # Собираем видео
    composite = CompositeVideoClip([
        bg,
        clip_resized.with_position("center"),
        caption
    ]).with_audio(clip.audio)


    clips.append(composite)

In [ ]:
clips.append(title_clip)

In [10]:
def make_transition(duration, w=1920, h=1080):
    return (
        ColorClip((w, h), color=(0, 0, 0), duration=duration)
        .with_effects([vfx.FadeIn(duration/2,0), vfx.FadeOut(duration/2,0)])
    )

transition = make_transition(FADE_DURATION)

In [11]:
final = concatenate_videoclips(
    clips,
    method="compose",
    transition=transition
)

final.write_videofile(
    OUTPUT_FILE,
    fps=30,
    codec="libx264",
    audio_codec="aac"
)

MoviePy - Building video ./result/output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ./result/output.mp4



frame_index:  63%|██████▎   | 1821/2911 [05:33<03:38,  4.99it/s, now=None]/home/sasha-sem/Projects/dobrokek-bot/.venv/lib/python3.12/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file ./export/video_files/Diddy_supply_runs_never_ends_#pranks_#funny_#funnyprank_#free.mp4, 691200 bytes wanted but 0 bytes read at frame index 1205 (out of a total 1206 frames), at time 40.17/40.21 sec. Using the last valid frame instead.
  warnings.warn(
                                                                          

MoviePy - Done !
MoviePy - video ready ./result/output.mp4
